In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Diplomatic2/Implementation")
!ls

 AGNN_1			    elliptic_txs_edgelist.csv   GRL_2.ipynb
 AGNN_2			    elliptic_txs_features.csv  'GRL_FINAL (4).ipynb'
 AGNN_3			    GAT_1		        GRL.ipynb
 data_balanced_1000.obj     GAT_2		        NoteBooks
 data_balanced_500.obj	    GAT_2_1		        SAGE_1
 data.obj		    GAT_3		        SAGE_2
 DGI			    GAT_FL_1		        SAGE_3
 elliptic_txs_classes.csv   GAT_TEST


In [ ]:
#PYTORCH INSTALL
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

In [ ]:
import pickle   
import torch
from torch_geometric.data import Data

fileObj = open('data.obj', 'rb')
data = pickle.load(fileObj)
fileObj.close()
print(data)

Data(x=[203769, 165], edge_index=[2, 234355], y=[203769], n_id=[203769], train_mask=[203769], val_mask=[203769], test_mask=[203769])


In [ ]:
#USE GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
data = data.to(device)


In [ ]:
#EDW NA DW ALLES PERIPTWSEIS GIA BATCHES!!!
#Epishs na dw th lista me tous geitones
from torch_geometric.loader import NeighborLoader

train_loader = NeighborLoader(data,num_neighbors=[-1, -1,-1], shuffle=True,batch_size=203769)

counter = 0
for batch in train_loader:

    print(len(batch.n_id))
    print(len(batch.y))
    print(type(batch))
    print(batch.test_mask)

203769
203769
<class 'torch_geometric.data.data.Data'>
tensor([False, False,  True,  ..., False, False, False], device='cuda:0')


In [ ]:
#SCRIPT 24
from torch_geometric.nn import DeepGraphInfomax, SAGEConv
import torch.nn.functional as F

class GraphSAGE(torch.nn.Module):
  """GraphSAGE"""
  def __init__(self):
    super().__init__()
    
    self.sage1 = SAGEConv(165, 128)
    self.sage2 = SAGEConv(128, 64)
    self.sage3 = SAGEConv(64, 64)
    
  def forward(self, x, edge_index):

      h = self.sage1(x, edge_index)
      h = torch.relu(h)
      h = F.dropout(h, p=0.5, training=self.training)
      h = self.sage2(h, edge_index)
      h = torch.relu(h)
      h = F.dropout(h, p=0.2, training=self.training)
      h = self.sage3(h, edge_index)
      h = torch.relu(h)

      return h

class SAGE_Classifier(torch.nn.Module):
  """GraphSAGE"""
  def __init__(self,encoder):
    super().__init__()

    self.encoder = encoder
    #self.encoder.requires_grad_(False)
    self.linear = torch.nn.Linear(64,2)
    self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)
    
  def forward(self, x, edge_index):

      h = self.encoder(x, edge_index)
      h = self.linear(h)

      return h, F.log_softmax(h, dim=1)

In [ ]:
#SCRIPT 25
from torch_geometric.nn import DeepGraphInfomax, GATConv
import torch.nn.functional as F

class GATmodel(torch.nn.Module):
    def __init__(self):
        super(GATmodel, self).__init__()
        self.hid = 64
        self.in_head = 8
      
        self.conv1 = GATConv(165, self.hid, heads=self.in_head, dropout=0.6)
        self.conv = GATConv(self.hid*self.in_head, self.hid, heads=self.in_head,concat=False, dropout=0.6)
        #self.conv2 = GATConv(self.hid*self.in_head, self.hid, heads=self.in_head, dropout=0.6)
       
          
    def forward(self,x, edge_index):
        
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv(x, edge_index)
        
        
        return x   


class GAT_Classifier(torch.nn.Module):
  def __init__(self,encoder):
    super().__init__()

    self.encoder = encoder
    self.encoder.requires_grad_(False)
    self.linear = torch.nn.Linear(64,2)
    self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)
    
  def forward(self, x, edge_index):

      h = self.encoder(x, edge_index)
      h = self.linear(h)

      return h, F.log_softmax(h, dim=1)

In [ ]:
#SCRIPT 26
from torch_geometric.nn import DeepGraphInfomax, AGNNConv
import torch.nn.functional as F

class AGNNmodel(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden = 128

        self.lin1 = torch.nn.Linear(165, self.hidden)
        self.prop1 = AGNNConv(requires_grad=False)
        self.prop2 = AGNNConv(requires_grad=True)


    def forward(self, x, edge_index):

        x = F.dropout(x, training=self.training)
        x = F.relu(self.lin1(x))
        x = self.prop1(x, edge_index)
        x = self.prop2(x, edge_index)

        return x 

class AGNN_Classifier(torch.nn.Module):
  def __init__(self,encoder):
    super().__init__()

    self.encoder = encoder
    self.encoder.requires_grad_(False)
    self.linear = torch.nn.Linear(128,2)
    self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)
    
  def forward(self, x, edge_index):

      h = self.encoder(x, edge_index)
      h = self.linear(h)

      return h, F.log_softmax(h, dim=1)        

In [ ]:
#DEBUGGING CODE WORKS!!!!
model1 = GraphSAGE()
print(model1)
model2 = SAGE_Classifier(model1)
print(model2)
best, trai_lo, val_lo = train2(model2,2,False)


In [ ]:
#DEBUGGING GAT WORKS
model3 = GATmodel()
print(model3)
model4 = GAT_Classifier(model3).to(device)
print(model4)
best, trai_lo, val_lo = train2(model4,2,False)


In [ ]:
#SCRIPT 16
#METRICS
!pip install torchmetrics
from torchmetrics.classification import AUROC
from torchmetrics.classification import BinaryPrecision
from torchmetrics.classification import BinaryRecall
from torchmetrics.classification import BinaryConfusionMatrix
from torchmetrics.classification import MulticlassRecall
from torchmetrics.classification import MulticlassPrecision
from torchmetrics.classification import MulticlassAUROC
from torchmetrics.classification import BinaryF1Score
from torchmetrics.classification import MulticlassF1Score

confmat = BinaryConfusionMatrix().to(device)
recall = MulticlassRecall(num_classes=2, average=None).to(device)
precision = MulticlassPrecision(num_classes=2, average=None).to(device)
aucroc = MulticlassAUROC(task="multiclass", num_classes=2).to(device)
metric = BinaryF1Score(average=None).to(device)
metric2 = MulticlassF1Score(num_classes=2, average=None).to(device)

def conf_matrix(pred_y, y):
  #confmat = BinaryConfusionMatrix().to(device)

  return confmat(pred_y, y).cpu().detach().numpy()

def brecall(pred_y, y):
  #recall = BinaryRecall().to(device)
  #recall = MulticlassRecall(num_classes=2, average=None).to(device)

  return recall(pred_y, y).cpu().detach().numpy()

def bprecision(pred_y, y):
  #precision = MulticlassPrecision(num_classes=2, average=None).to(device)

  return precision(pred_y, y).cpu().detach().numpy()

def auc_roc(pred_y, y):
  #aucroc = AUROC(task="binary").to(device)
  #print('AUC ROC',pred_y[:10])
  #print('AUC ROC',y.shape)
  #aucroc = MulticlassAUROC(task="multiclass", num_classes=2).to(device)

  return aucroc(pred_y, y).cpu().detach().numpy()

def f1score(pred_y, y):
  #metric = BinaryF1Score(average=None).to(device)

  return metric2(pred_y, y).cpu().detach().numpy()


def accuracy(pred_y, y):
    
    return ((pred_y == y).sum() / len(y)).item() 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 25.9 MB/s eta 0:00:00


In [ ]:
# SCRIPT 18

def train2(model, epochs,yes_no):
    weigths = [2.5, 0.625]

    if yes_no == True:

      wei = torch.FloatTensor(weigths).to(device)
      print('WEIGTHS = ',wei)
      criterion = torch.nn.CrossEntropyLoss(weight=wei)
    else:
      criterion = torch.nn.CrossEntropyLoss()

    optimizer = model.optimizer
    
    model.train()

    train_loss_total = []
    val_loss_total = []
    best_model = model
    best_val_loss = 1000.0
    
    for epoch in range(epochs+1):
      total_loss = 0
      total_val_loss = 0
      acc = 0
      val_loss = 0
      val_acc = 0
      auc_roc_train = 0
      auc_roc_val = 0 
      pre_train = 0 
      pre_val = 0
      recall_train = 0
      recall_val = 0
      f1_train = 0
      f1_val = 0
 
      # Train on batches
      
      optimizer.zero_grad()
        
      logit, out = model(data.x, data.edge_index)
      loss = criterion(logit[data.train_mask], data.y[data.train_mask])
      total_loss += float(loss)

      acc += accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
      auc_roc_train += auc_roc(out[data.train_mask], data.y[data.train_mask])
      pre_train += bprecision(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
      recall_train += brecall(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
      f1_train += f1score(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
      conf_matrix_train = conf_matrix(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
      print(conf_matrix_train)
      loss.backward()
      optimizer.step()

        # Validation
      with torch.no_grad():  
        val_loss = criterion(logit[data.val_mask], data.y[data.val_mask])
        total_val_loss += float(val_loss)
        print('TRAIN LOSS', total_loss)
        print('VALIDATION LOSS', total_val_loss)

        val_acc += accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
        auc_roc_val += auc_roc(out[data.val_mask], data.y[data.val_mask])
        pre_val += bprecision(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
        recall_val += brecall(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
        f1_val += f1score(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
        conf_matrix_val = conf_matrix(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
        print(conf_matrix_val)

        print("Train accuracy = ", acc, "Val accuracy = ", val_acc)
        print("AUC ROC SCORE TRAIN = ",auc_roc_train,"AUC ROC SCORE VAL = ", auc_roc_val)
        print("precision SCORE TRAIN = ",pre_train,"precision SCORE VAL = ", pre_val)
        print("RECALL SCORE TRAIN = ",recall_train,"RECALL SCORE VAL = ", recall_val)
        print("F1 SCORE TRAIN = ",f1_train,"F1 SCORE VAL = ", f1_val)

        if(total_val_loss < best_val_loss):
          best_val_loss = total_val_loss
          best_model = model
          print("NEW BEST at epoch ", epoch)
        train_loss_total.append(total_loss)
        val_loss_total.append(total_val_loss)  

    return best_model, train_loss_total, val_loss_total 

In [ ]:
#SCRIPT 17
def train(model,epochs,yes_no):

    weigths = [2.5, 0.625]

    if yes_no == True:

      wei = torch.FloatTensor(weigths).to(device)
      print('WEIGTHS = ',wei)
      criterion = torch.nn.CrossEntropyLoss(weight=wei)
    else:
      criterion = torch.nn.CrossEntropyLoss()

    #criterion = torch.nn.CrossEntropyLoss()
    #criterion2 = torch.nn.NLLLoss()
    optimizer = model.optimizer

    train_loss_total = []
    val_loss_total = []
    best_model = model
    best_val_loss = 1000.0

    model.train()
    for epoch in range(epochs+1):
      total_loss = 0
      total_val_loss = 0
      acc = 0
      val_loss = 0
      val_acc = 0
      auc_roc_train = 0
      auc_roc_val = 0 
      pre_train = 0 
      pre_val = 0
      recall_train = 0
      recall_val = 0
      f1_train = 0
      f1_val = 0
      nll_loss_train = 0
      nll_loss_val = 0
 
      # Train on batches
      for batch in train_loader:
        optimizer.zero_grad()
        
        logit, out = model(batch.x, batch.edge_index)
        loss = criterion(logit[batch.train_mask], batch.y[batch.train_mask])
        total_loss += float(loss)

        #nll_loss_train = criterion2(out[batch.train_mask], batch.y[batch.train_mask])
        #nll_loss_val = criterion2(out[batch.val_mask], batch.y[batch.val_mask])


        acc += accuracy(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
        auc_roc_train += auc_roc(out[batch.train_mask], batch.y[batch.train_mask])
        pre_train += bprecision(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
        recall_train += brecall(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
        f1_train += f1score(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
        conf_matrix_train = conf_matrix(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])

        print(conf_matrix_train)

        loss.backward()
        optimizer.step()

        # Validation
        with torch.no_grad():
          val_loss = criterion(logit[batch.val_mask], batch.y[batch.val_mask])
          total_val_loss += float(val_loss)

          val_acc += accuracy(out[batch.val_mask].argmax(dim=1), batch.y[batch.val_mask])
          auc_roc_val += auc_roc(out[batch.val_mask], batch.y[batch.val_mask])
          pre_val += bprecision(out[batch.val_mask].argmax(dim=1), batch.y[batch.val_mask])
          recall_val += brecall(out[batch.val_mask].argmax(dim=1), batch.y[batch.val_mask])
          f1_val += f1score(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
          conf_matrix_val = conf_matrix(out[batch.val_mask].argmax(dim=1), batch.y[batch.val_mask])

          #print(conf_matrix_val)
          print('TRAIN LOSS', total_loss)
          print('VALIDATION LOSS', total_val_loss)

          print("Train accuracy = ", acc/len(train_loader), "Val accuracy = ", val_acc/len(train_loader))
          print("AUC ROC SCORE TRAIN = ",auc_roc_train/len(train_loader),"AUC ROC SCORE VAL = ", auc_roc_val/len(train_loader))
          print("precision SCORE TRAIN = ",pre_train/len(train_loader),"precision SCORE VAL = ", pre_val/len(train_loader))
          print("RECALL SCORE TRAIN = ",recall_train/len(train_loader),"RECALL SCORE VAL = ", recall_val/len(train_loader))
          print("F1 SCORE TRAIN = ",f1_train/len(train_loader),"F1 SCORE VAL = ", f1_val/len(train_loader))

        if(total_val_loss < best_val_loss):
          best_val_loss = total_val_loss
          best_model = model
          print("NEW BEST at epoch ", epoch)
        train_loss_total.append(total_loss)
        val_loss_total.append(total_val_loss)
      
    return best_model, train_loss_total, val_loss_total

def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    with torch.no_grad():
      model.eval()
      _, out = model(data.x, data.edge_index)
      acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
      auc_roc_acc = auc_roc(out[data.test_mask], data.y[data.test_mask])
      precision_score = bprecision(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
      recall_score = brecall(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
      f1_test = f1score(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
      conf_mat = conf_matrix(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])


    return acc, auc_roc_acc, precision_score, recall_score,f1_test, conf_mat


In [ ]:
#SCRIPT27
#TRAIN ENCODER WITH DGI
#encoder_SAGE = GraphSAGE()
#print(encoder_SAGE)
#encoder_GAT = GATmodel()
#print(encoder_GAT)
#encoder_AGNN = AGNNmodel()
#print(encoder_AGNN)

def corruption(x, edge_index):
    return x[torch.randperm(x.size(0))], edge_index

model = DeepGraphInfomax(hidden_channels=128, encoder=AGNNmodel(),summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),corruption=corruption).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_dgi():
    model.train()
    optimizer.zero_grad()
    pos_z, neg_z, summary = model(data.x, data.edge_index)
    loss = model.loss(pos_z, neg_z, summary)
    loss.backward()
    optimizer.step()
    return loss.item()


In [ ]:
print(model.encoder)

In [ ]:
for epoch in range(1,200):
  loss = train_dgi()
  print(loss)

In [ ]:
#BUILT IN CLASSIFIER
model.eval()
z, _, _ = model(data.x, data.edge_index)
acc = model.test(z[data.train_mask], data.y[data.train_mask], z[data.test_mask], data.y[data.test_mask], max_iter=150)
print(acc)

0.9176300578034682


In [ ]:
#TRAIN AGNN CLASSIFIER
#TRAIN SAGE CLASSIFIER
AGNN_Class = AGNN_Classifier(model.encoder).to(device)
print(AGNN_Class)
best, trai_lo, val_lo = train2(AGNN_Class,200,False)
acc_SAGE, auc_roc_acc_SAGE, precision_score_SAGE, recall_score_SAGE, f1_test_SAGE, conf_mat_SAGE = test(best,data)

In [ ]:
#TRAIN GAT CLASSIFIER
GAT_Class = GAT_Classifier(model.encoder).to(device)
print(GAT_Class)
best, trai_lo, val_lo = train2(GAT_Class,200,False)
acc_GAT, auc_roc_acc_GAT, precision_score_GAT, recall_score_GAT, f1_test_GAT, conf_mat_GAT = test(best,data)

In [ ]:
#TRAIN SAGE CLASSIFIER
SAGE_Class = SAGE_Classifier(model.encoder).to(device)
print(SAGE_Class)
best, trai_lo, val_lo = train(SAGE_Class,200,False)
acc_SAGE, auc_roc_acc_SAGE, precision_score_SAGE, recall_score_SAGE, f1_test_SAGE, conf_mat_SAGE = test(best,data)

In [ ]:
os.chdir("/content/drive/MyDrive/Diplomatic2/Implementation/DGI")
model = 'AGNN_Frozen.model'
#model = 'SAGE_Frozen.model'
file2save_everything = open(model,'wb')
#pickle.dump(best_model, file2save_everything)
pickle.dump(trai_lo, file2save_everything)
pickle.dump(val_lo, file2save_everything)
pickle.dump(acc_SAGE, file2save_everything)
pickle.dump(auc_roc_acc_SAGE, file2save_everything)
pickle.dump(precision_score_SAGE, file2save_everything)
pickle.dump(recall_score_SAGE, file2save_everything)
pickle.dump(f1_test_SAGE, file2save_everything)
pickle.dump(conf_mat_SAGE, file2save_everything)

file2save_everything.close()

In [ ]:
#EVAL CLASSIFIER
acc, auc_roc_acc, precision, recall_score, f1_test, conf_mat = test(best,data)
print(acc, auc_roc_acc, precision, recall_score, f1_test)

0.7574508190155029 0.9358834 [0.24971001 0.9913998 ] [0.93045187 0.7414514 ] [0.39374793 0.84839916]


In [ ]:
#SAVE EVERYTHING FOR COMPARISON
import pickle

os.chdir("/content/drive/MyDrive/Diplomatic2/Implementation")

a = os.path.isdir('DGI')
if a == False:
  os.mkdir("DGI")

os.chdir("/content/drive/MyDrive/Diplomatic2/Implementation/DGI")

architecture = 'AGNN.model'

file2save_everything = open(architecture,'wb')
#pickle.dump(best_model, file2save_everything)
pickle.dump(trai_lo, file2save_everything)
pickle.dump(val_lo, file2save_everything)
pickle.dump(acc, file2save_everything)
pickle.dump(auc_roc_acc, file2save_everything)
pickle.dump(precision, file2save_everything)
pickle.dump(recall_score, file2save_everything)
pickle.dump(f1_test, file2save_everything)
pickle.dump(conf_mat, file2save_everything)

file2save_everything.close()